In [14]:
## first block of code is just importing the requirements of the project

import sys 
import os
import h5py
import pickle
from matplotlib import pyplot as plt

from scipy import ndimage as ndi
from scipy import stats as sstats

import math
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from random import shuffle
from tqdm import tqdm, tnrange, tqdm_notebook
import collections
import random
import mahotas 
from sklearn.preprocessing import normalize
from PIL import Image
import imutils
import logging

from glob import glob

In [15]:
# this block of code is importing all the various parts of the project from their respective modules
from icon_util_v2 import *
from methods_v2 import *
from aberrations import *

In [16]:
hdf5_file = h5py.File('LLD-icon.hdf5', 'r')
images, _ = (hdf5_file['data'], hdf5_file['labels/resnet/rc_64'])
images = images[:100]
# transpose the images because they're stored in a weird color channel first format, as indicated by shape[0] being 3
images = [np.transpose(i) if i.shape[0] == 3 else i for i in images]
print(len(images))

100


In [4]:
image_set_name = "icon100"
paramsets=[{},{"method":"contour"},{"center":False},{"method":"contour","center":False}, {"method":"contour","center":True,"hulls":True}]
funparampair=zip([split_method, symmetric_best_split_method], paramsets)

logdir = "Logs20210625_first"
if not os.path.exists("Logs"):
    os.mkdir("Logs")
if not os.path.exists("Logs/"+logdir):
        os.mkdir("Logs/"+logdir)

method_classes = [zernike_method, orb_method, neural_method, small_neural_method]
#method_classes = [neural_method, small_neural_method]
methods = [symmetric_best_split_method(m,0) for m in method_classes] + [symmetric_best_split_method(m,1) for m in method_classes] + [symmetric_best_split_method(m,2) for m in method_classes]

create_databases(images,methods,image_set_name)
run_in_chunks2(methods, images, aberrations, chunk_size=100, weights=[],          
    logdir=logdir)
print("Done")
#method_classes = [zernike_method, orb_method, neural_method, small_neural_method]
#methods=[m() for m in method_classes]

0
50
0
50
0
50
0
50
0
50
0
50
0
50
0
50
0
50
0
50
0
50
0
50
Chunk: 1
0


NameError: name 'scipy' is not defined

In [10]:
img=images[4]
img2=prep_img(img, False)
contours,edges=find_contours(img2)

crops = []
for idx in range(len(contours)):  
    mask = np.zeros_like(img2) # Create mask where white is what we want, black otherwise
    cv2.drawContours(mask, contours, idx, (255,255,255), -1) # Draw filled contour in mask

    # Now crop
    (y, x, z) = np.where(mask == 255)
    (topy, topx) = (np.min(y), np.min(x))
    (bottomy, bottomx) = (np.max(y), np.max(x))
    print(bottomx, topx, bottomy, topy)
    out = img2[topy:bottomy+1, topx:bottomx+1]
    print(out.shape)
    out = cv2.resize(out,img2.shape[:-1],interpolation=cv2.INTER_AREA)
    print(out.shape)
    crops.append(out)

    
for im in crops:
    # Show the output image
    cv2.imshow('Output', im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

89 33 97 30
(68, 57, 3)
(128, 128, 3)
80 36 95 35
(61, 45, 3)
(128, 128, 3)
81 58 86 77
(10, 24, 3)
(128, 128, 3)
85 51 81 72
(10, 35, 3)
(128, 128, 3)
46 36 95 85
(11, 11, 3)
(128, 128, 3)


In [ ]:
cv2.imshow("output", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

[array([[[ 7,  0]],
 
        [[ 6,  1]],
 
        [[ 5,  2]],
 
        [[ 4,  3]],
 
        [[ 3,  4]],
 
        [[ 2,  5]],
 
        [[ 2,  6]],
 
        [[ 1,  7]],
 
        [[ 1,  8]],
 
        [[ 0,  9]],
 
        [[ 0, 10]],
 
        [[ 0, 11]],
 
        [[ 0, 12]],
 
        [[ 0, 13]],
 
        [[ 0, 14]],
 
        [[ 0, 15]],
 
        [[ 0, 16]],
 
        [[ 0, 17]],
 
        [[ 0, 18]],
 
        [[ 1, 19]],
 
        [[ 1, 20]],
 
        [[ 2, 21]],
 
        [[ 2, 22]],
 
        [[ 3, 23]],
 
        [[ 4, 24]],
 
        [[ 4, 25]],
 
        [[ 3, 26]],
 
        [[ 2, 26]],
 
        [[ 1, 27]],
 
        [[ 0, 28]],
 
        [[ 0, 29]],
 
        [[ 0, 30]],
 
        [[ 0, 31]],
 
        [[ 1, 31]],
 
        [[ 2, 31]],
 
        [[ 2, 30]],
 
        [[ 2, 29]],
 
        [[ 3, 28]],
 
        [[ 4, 29]],
 
        [[ 5, 30]],
 
        [[ 5, 31]],
 
        [[ 6, 31]],
 
        [[ 7, 31]],
 
        [[ 7, 30]],
 
        [[ 8, 29]],
 
        [[

In [17]:
image_set_name = "icon100"
logdir = "Logs20210625_first"
if not os.path.exists("Logs"):
    os.mkdir("Logs")
if not os.path.exists("Logs/"+logdir):
        os.mkdir("Logs/"+logdir)
        
method_classes = [zernike_method, orb_method, neural_method, small_neural_method]
paramsets=[{},{"method":"contour"},{"center":False},{"method":"contour","center":False}, {"method":"contour","center":True,"hulls":True}]
for fun in [split_method, symmetric_best_split_method]:
    for param in paramsets:
        
        methods = [fun(m,**param) for m in method_classes]
        create_databases(images,methods,image_set_name)
        run_in_chunks2(methods, images, aberrations, chunk_size=100, weights=[],          
            logdir=logdir)
        print("Done "+str(param))

0
50
0
50
0
50
0
50
Chunk: 1
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
0
10
20
30
40
50
60
70
80
90
Done {}


TypeError: __init__() got an unexpected keyword argument 'method'